# Credit and Debit Card Fraud Detection

## Modeling

## Table Of Contents: <a class="anchor" id="mtop"></a>

- [Fraud classification](#mUnclassification)
- [Under Sampling Data balancing](#mUndataBalancing)
- [Fraud Classification with Under Sampling](#mUnclassification)
    * [Feature Selection](#mUnfeatureSelection)
    * [Decision Trees models](#mUndecisionTrees)
    * [Random Forests models](#mUnrandomForests)
    * [AdaBoost model](#mUnAdaBoost)
    * [Suport Vector Machines models](#mUnSVM)
    * [Artificial Neural Networks](#mUnANN)
- [Upper-sampling data balancing](#mUpdataBalancing)
- [Fraud Classification with Upper Sampling](#mUpclassification)
    * [Feature Selection](#mUpfeatureSelection)
    * [Decision Trees models](#mUpdecisionTrees)
    * [Random Forests models](#mUprandomForests)
    * [AdaBoost model](#mUpAdaBoost)
    * [Suport Vector Machines models](#mUpSVM)
    * [Artificial Neural Networks](#mUpANN)    
- [Conclusions](#mconclusions)

In [22]:
import pandas as pd
from imblearn.under_sampling import RandomUnderSampler
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import RFECV
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn import svm
from sklearn.ensemble import AdaBoostClassifier
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import classification_report
from sklearn.ensemble import AdaBoostClassifier
from sklearn import svm
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score

# Fraud classification <a class="anchor" id="mUnclassification"></a>

[TOC](#mtop)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
#Importing the dataset used to train and test the machine learning models
#fraud = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/preparedFraud.csv')
fraud = pd.read_csv('..\CreditCardFraudDetectionData\preparedFraud2.csv')

In [3]:
X=fraud[['TransactionAmt','card1','addr1','addr2',
        'C1','C2','C3','C4','C5','C6','C7','C8','C9','C10','C11','C12','C13','C14',
        'V95', 'V96', 'V97','V98', 'V99', 'V100', 'V101', 'V102', 'V103', 'V104', 'V105', 'V106', 'V107', 'V108', 'V109', 'V110', 'V111',
        'V112', 'V113', 'V114', 'V115', 'V116', 'V117', 'V118', 'V119', 'V120', 'V121', 'V122', 'V123', 'V124', 'V125', 'V126' , 'V127',
        'V128', 'V129', 'V130', 'V131', 'V132', 'V133', 'V134', 'V135', 'V136', 'V137']]

y=fraud['isFraud']

## Under-sampling data balancing  <a class="anchor" id="mUndataBalancing"></a>

[TOC](#mtop)

The imbalanced datasets are a characteristic of the problem of credit card fraud detection. One can see that the fraud dataset is imbalanced. The issue presents much more legitimate transactions than fraudulent transactions. It is not different in this case, so we must use a technique to balance the data. As the quantity of data is enough, one can use the under-sampling technique. After this process, the remaining data has over 10 thousand legitimate and fraudulent transactions.

In [4]:
#Let's perform an undersampling to balance the data. In this case, it was used a technique of random sampling with replacement.
rus = RandomUnderSampler(random_state=42, replacement=True)# fit predictor and target variable
x_rus, y_rus = rus.fit_resample(X, y)

In [5]:
X_train, X_test, y_train, y_test = train_test_split(x_rus, y_rus,
                                                    test_size=0.2,
                                                    random_state=42)

This section presents some baseline models to set the stage for advanced modeling in the future project step. One can see the performance of four models: Decision Trees, Random Forests, Adaboost,  and Support Vector Machines. A feature selection is set for all models using a recursive feature elimination with cross-validation based on the Random Forests classifier. Next, a pipeline is designed with these steps: feature selection, feature scaling, model hyperparameters optimization, and training. To articulate a model evaluation, a performance metric report for each model is presented with accuracy, precision, recall, and f1-score.

### Feature Selection  <a class="anchor" id="mUnfeatureSelection"></a>

[TOC](#mtop)

In [12]:
min_features_to_select = 1  # Minimum number of features to consider
clf = RandomForestClassifier(n_estimators=10,
                             random_state=42)

cv = StratifiedKFold(5)

rfecv = RFECV(
    estimator=clf,
    step=1,
    cv=cv,
    scoring="accuracy",
    min_features_to_select=min_features_to_select,
    n_jobs=2,
)
rfecv.fit(X_train, y_train)

print(f"Optimal number of features: {rfecv.n_features_}")
print(f"Selected features: {X_train.columns[rfecv.support_]}")

Optimal number of features: 13
Selected features: Index(['TransactionAmt', 'card1', 'addr1', 'C1', 'C2', 'C5', 'C9', 'C13',
       'C14', 'V126', 'V127', 'V128', 'V133'],
      dtype='object')


### Decision trees models <a class="anchor" id="mUndecisionTrees"></a>

[TOC](#mtop)

In [24]:
#this is the classifier used for feature selection
clf_featr_sele = RandomForestClassifier(n_estimators=10,
                             random_state=42)

rfecv = RFECV(estimator=clf_featr_sele,
              step=1,
              cv=5,
              scoring = 'roc_auc')

#you can have different classifier for your final classifier
clf = DecisionTreeClassifier()


CV_dt = GridSearchCV(clf,
                      param_grid={
                          'criterion': ['gini', 'entropy'],
                          'max_depth': [2,4,6],
                          'min_samples_leaf': [1, 2]},
                      cv= 5,
                      scoring = 'roc_auc')


pipeline  = Pipeline([('feature_sele',rfecv),
                      ('scale', StandardScaler()),
                      ('clf_cv',CV_dt)])

pipeline.fit(X_train, y_train)
y_pred=pipeline.predict(X_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.72      0.78      0.75      2056
           1       0.77      0.70      0.73      2106

    accuracy                           0.74      4162
   macro avg       0.74      0.74      0.74      4162
weighted avg       0.74      0.74      0.74      4162



The Decision Trees performance was:
- Accuracy equals to 74%.
- Precision equals 72% for legitimate class and 77% for fraudulent class.
- Recall equals 78% for genuine class and 70% for fraudulent class.
- F1-score equals 75% for the genuine class and 73% for the fraudulent class.

### Random forests models<a class="anchor" id="mUnrandomForests"></a>

[TOC](#mtop)

In [25]:
#this is the classifier used for feature selection
clf_featr_sele = RandomForestClassifier(n_estimators=10,
                             random_state=42)

rfecv = RFECV(estimator=clf_featr_sele,
              step=1,
              cv=5,
              scoring = 'roc_auc')

#you can have different classifier for your final classifier
clf = RandomForestClassifier(n_estimators=10,
                             random_state=42,
                             class_weight="balanced")
CV_rfc = GridSearchCV(clf,
                      param_grid={'max_depth':[2,3]},
                      cv= 5, scoring = 'roc_auc')


pipeline  = Pipeline([('feature_sele',rfecv),
                      ('scale', StandardScaler()),
                      ('clf_cv',CV_rfc)])

pipeline.fit(X_train, y_train)
y_pred=pipeline.predict(X_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.69      0.79      0.74      2056
           1       0.76      0.65      0.70      2106

    accuracy                           0.72      4162
   macro avg       0.73      0.72      0.72      4162
weighted avg       0.73      0.72      0.72      4162



### AdaBoost model <a class="anchor" id="mUnAdaBoost"></a>

[TOC](#mtop)

In [27]:
#this is the classifier used for feature selection
clf_featr_sele = RandomForestClassifier(n_estimators=10,
                             random_state=42)

rfecv = RFECV(estimator=clf_featr_sele,
              step=1,
              cv=5,
              scoring = 'roc_auc')

#you can have different classifier for your final classifier
DTC = DecisionTreeClassifier(class_weight = "balanced")
clf = AdaBoostClassifier()

CV_ada = GridSearchCV(clf,
                      param_grid = {
                          'estimator':[DTC],
                          'learning_rate': [0.1, 0.5],
                          'n_estimators': [10, 20]},
                      cv= 5,
                      scoring = 'roc_auc')


pipeline  = Pipeline([('feature_sele',rfecv),
                      ('scale', StandardScaler()),
                      ('clf_cv',CV_ada)])

pipeline.fit(X_train, y_train)
y_pred=pipeline.predict(X_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.77      0.79      0.78      2056
           1       0.79      0.77      0.78      2106

    accuracy                           0.78      4162
   macro avg       0.78      0.78      0.78      4162
weighted avg       0.78      0.78      0.78      4162



The Adaboost performance was:
- Accuracy equals to 78%.
- Precision equals 78% for the legitimate class and 78% for the fraudulent class.
- Recall equals 78% for genuine class and 79% for fraudulent class.
- F1-score equals 78% for the genuine class and 78% for the fraudulent class.

### Suport Vector Machines model <a class="anchor" id="mUnSVM"></a>

[TOC](#mtop)

In [29]:
#this is the classifier used for feature selection
clf_featr_sele = RandomForestClassifier(n_estimators=10,
                             random_state=42)

rfecv = RFECV(estimator=clf_featr_sele,
              step=1,
              cv=5,
              scoring = 'roc_auc')

#you can have different classifier for your final classifier
clf = svm.SVC()

CV_svm = GridSearchCV(clf,
                      param_grid={'C':[0.1, 1.0, 10],
                                  'kernel':['linear','rbf']},
                      cv= 5,
                      scoring = 'roc_auc')


pipeline  = Pipeline([('feature_sele',rfecv),
                      ('scale', StandardScaler()),
                      ('clf_cv',CV_svm)])

pipeline.fit(X_train, y_train)
y_pred=pipeline.predict(X_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.69      0.81      0.74      2056
           1       0.77      0.64      0.70      2106

    accuracy                           0.72      4162
   macro avg       0.73      0.73      0.72      4162
weighted avg       0.73      0.72      0.72      4162



The Support Vector Machines performance was:
- Accuracy equals to 72%.
- Precision equals 69% for the legitimate class and 77% for the fraudulent class.
- Recall equals 81% for genuine class and 64% for fraudulent class.
- F1-score equals 74% for the genuine class and 70% for the fraudulent class.

### Artificial Neural Networks <a class="anchor" id="mUnANN"></a>

[TOC](#mtop)

In [13]:
class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.hidden1 = nn.Linear(rfecv.n_features_, 12)
        self.act1 = nn.ReLU()
        self.hidden2 = nn.Linear(12, 8)
        self.act2 = nn.ReLU()
        self.output = nn.Linear(8, 1)
        self.act_output = nn.Sigmoid()

    def forward(self, x):
        x = self.act1(self.hidden1(x))
        x = self.act2(self.hidden2(x))
        x = self.act_output(self.output(x))
        return x

model = Net()
print(model)

Net(
  (hidden1): Linear(in_features=13, out_features=12, bias=True)
  (act1): ReLU()
  (hidden2): Linear(in_features=12, out_features=8, bias=True)
  (act2): ReLU()
  (output): Linear(in_features=8, out_features=1, bias=True)
  (act_output): Sigmoid()
)


In [14]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
model.to(device)
print(device)

cpu


In [15]:
loss_fn = nn.BCELoss()  # binary cross entropy
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [16]:
scaler = StandardScaler().fit(X_train[X_train.columns[rfecv.support_]])
X_tr_scaled = scaler.transform(X_train[X_train.columns[rfecv.support_]])
X_te_scaled = scaler.transform(X_test[X_test.columns[rfecv.support_]])

In [17]:
X_train_nn = torch.tensor(X_tr_scaled, dtype=torch.float32).to(device)
y_train_nn = torch.tensor(y_train.to_numpy(), dtype=torch.float32).reshape(-1, 1).to(device)

In [18]:
n_epochs = 25 #50
batch_size = 10

for epoch in range(n_epochs):
    for i in range(0, len(X_train_nn), batch_size):
        Xbatch = X_train_nn[i:i+batch_size]
        y_pred = model(Xbatch)
        ybatch = y_train_nn[i:i+batch_size]
        loss = loss_fn(y_pred, ybatch)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    print(f'Finished epoch {epoch}, latest loss {loss}')

Finished epoch 0, latest loss 0.5931800603866577
Finished epoch 1, latest loss 0.5147647857666016
Finished epoch 2, latest loss 0.4805735647678375
Finished epoch 3, latest loss 0.4662722647190094
Finished epoch 4, latest loss 0.45030370354652405
Finished epoch 5, latest loss 0.4413246214389801
Finished epoch 6, latest loss 0.436825156211853
Finished epoch 7, latest loss 0.4327261447906494
Finished epoch 8, latest loss 0.42343202233314514
Finished epoch 9, latest loss 0.4231400191783905
Finished epoch 10, latest loss 0.4182569682598114
Finished epoch 11, latest loss 0.40914949774742126
Finished epoch 12, latest loss 0.4110928475856781
Finished epoch 13, latest loss 0.41539740562438965
Finished epoch 14, latest loss 0.4185698926448822
Finished epoch 15, latest loss 0.42534133791923523
Finished epoch 16, latest loss 0.41759946942329407
Finished epoch 17, latest loss 0.4302314519882202
Finished epoch 18, latest loss 0.4158015549182892
Finished epoch 19, latest loss 0.42040184140205383
Fini

In [24]:
X_test_nn = torch.tensor(X_te_scaled, dtype=torch.float32).to(device)
y_test_nn = torch.tensor(y_test.to_numpy(), dtype=torch.float32).reshape(-1, 1).to(device)

In [25]:
with torch.no_grad(): y_pred = model(X_test_nn)
accuracy = (y_pred.round() == y_test_nn).float().mean()
print(f"Accuracy {accuracy}")

Accuracy 0.7426717877388


In [26]:
y_pred=pd.DataFrame(y_pred.round().cpu().detach().numpy())
print(f"Precision {precision_score(y_test, y_pred, pos_label=1)}")
print(f"Recall {recall_score(y_test, y_pred, pos_label=1)}")
print(f"F1-score {f1_score(y_test, y_pred, pos_label=1)}")

Precision 0.7841845140032949
Recall 0.6780626780626781
F1-score 0.7272727272727273


In [27]:
print(f"Precision {precision_score(y_test, y_pred, pos_label=0)}")
print(f"Recall {recall_score(y_test, y_pred, pos_label=0)}")
print(f"F1-score {f1_score(y_test, y_pred, pos_label=0)}")

Precision 0.7103801794105084
Recall 0.808852140077821
F1-score 0.7564248351148511


<table>
    <thead>
        <tr>
            <th>Models</th>
            <th>Accuracy</th>
            <th colspan=2 align=center>Precision</th>
            <th colspan=2 align=center>Recall</th>
            <th colspan=2 align=center>F1-score</th>
        </tr>
    </thead>
    <tbody>
        <tr>
            <td></td>       
            <td></td>
            <td>Legitimate</td>
            <td>Fraud</td>
            <td>Legitimate</td>
            <td>Fraud</td>
            <td>Legitimate</td>
            <td>Fraud</td>
        </tr>
        <tr>
            <td align=right>Decision trees</td>
            <td  align=center>3</td>            
            <td align=center>sss</td>
            <td align=center>zz</td>
            <td align=center>aa</td>
            <td align=center>ss</td>
            <td align=center>ss</td>
            <td align=center>ss</td>
        </tr>
        <tr>
            <td align=right>Random forests</td>
            <td align=center>qq</td>
            <td align=center>ww</td>
            <td align=center>zz</td>
            <td align=center></td>
            <td align=center>ss</td>
            <td align=center>dd</td>
            <td align=center>ss</td>
        </tr>
        <tr>
            <td align=right>Adaboost</td>
            <td align=center>ll</td>
            <td align=center>xx</td>
            <td align=center>zz</td>
            <td align=center></td>
            <td align=center>ss</td>
            <td align=center></td>
            <td align=center>ss</td>
        </tr>
        <tr>
            <td align=right>Suport vector machines</td>
            <td align=center>ll</td>
            <td align=center>oo</td>
            <td align=center>zz</td>
            <td align=center>dd</td>
            <td align=center>ss</td>
            <td align=center></td>
            <td align=center>ss</td>
        </tr>
        <tr>
            <td align=right>Artificial neural networks</td>
            <td align=center>L2</td>
            <td align=center>L3</td>
            <td align=center>zz</td>
            <td align=center></td>
            <td align=center>ss</td>
            <td align=center></td>
            <td align=center>ss</td>
        </tr>
    </tbody>
</table>

### Upper-sampling data balancing  <a class="anchor" id="mUpdataBalancing"></a>

[TOC](#mtop)

In [28]:
from imblearn.over_sampling import SMOTE
X_resampled, y_resampled = SMOTE().fit_resample(X, y)

### Fraud Classification with Upper Sampling  <a class="anchor" id="mUpclassification"></a>

[TOC](#mtop)

In [29]:
X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_resampled,
                                                    test_size=0.2,
                                                    random_state=42)

In [30]:
min_features_to_select = 1  # Minimum number of features to consider
clf = RandomForestClassifier(n_estimators=10,
                             random_state=42)

cv = StratifiedKFold(5)

rfecv = RFECV(
    estimator=clf,
    step=1,
    cv=cv,
    scoring="accuracy",
    min_features_to_select=min_features_to_select,
    n_jobs=2,
)
rfecv.fit(X_train, y_train)

print(f"Optimal number of features: {rfecv.n_features_}")
print(f"Selected features: {X_train.columns[rfecv.support_]}")

KeyboardInterrupt: 

### Feature Selection  <a class="anchor" id="mUpfeatureSelection"></a>

[TOC](#mtop)

### Decision trees models <a class="anchor" id="mUpdecisionTrees"></a>

[TOC](#mtop)

In [ ]:
#this is the classifier used for feature selection
clf_featr_sele = RandomForestClassifier(n_estimators=10,
                             random_state=42)

rfecv = RFECV(estimator=clf_featr_sele,
              step=1,
              cv=5,
              scoring = 'roc_auc')

#you can have different classifier for your final classifier
clf = DecisionTreeClassifier()


CV_dt = GridSearchCV(clf,
                      param_grid={
                          'criterion': ['gini', 'entropy'],
                          'max_depth': [2,4,6],
                          'min_samples_leaf': [1, 2]},
                      cv= 5,
                      scoring = 'roc_auc')


pipeline  = Pipeline([('feature_sele',rfecv),
                      ('scale', StandardScaler()),
                      ('clf_cv',CV_dt)])

pipeline.fit(X_train, y_train)
y_pred=pipeline.predict(X_test)
print(classification_report(y_test, y_pred))

### Random forests models<a class="anchor" id="mUprandomForests"></a>

[TOC](#mtop)

In [ ]:
#this is the classifier used for feature selection
clf_featr_sele = RandomForestClassifier(n_estimators=10,
                             random_state=42)

rfecv = RFECV(estimator=clf_featr_sele,
              step=1,
              cv=5,
              scoring = 'roc_auc')

#you can have different classifier for your final classifier
clf = RandomForestClassifier(n_estimators=10,
                             random_state=42,
                             class_weight="balanced")
CV_rfc = GridSearchCV(clf,
                      param_grid={'max_depth':[2,3]},
                      cv= 5, scoring = 'roc_auc')


pipeline  = Pipeline([('feature_sele',rfecv),
                      ('scale', StandardScaler()),
                      ('clf_cv',CV_rfc)])

pipeline.fit(X_train, y_train)
y_pred=pipeline.predict(X_test)
print(classification_report(y_test, y_pred))

### AdaBoost model <a class="anchor" id="mUpAdaBoost"></a>

[TOC](#mtop)

In [ ]:
#this is the classifier used for feature selection
clf_featr_sele = RandomForestClassifier(n_estimators=10,
                             random_state=42)

rfecv = RFECV(estimator=clf_featr_sele,
              step=1,
              cv=5,
              scoring = 'roc_auc')

#you can have different classifier for your final classifier
DTC = DecisionTreeClassifier(class_weight = "balanced")
clf = AdaBoostClassifier()

CV_ada = GridSearchCV(clf,
                      param_grid = {
                          'estimator':[DTC],
                          'learning_rate': [0.1, 0.5],
                          'n_estimators': [10, 20]},
                      cv= 5,
                      scoring = 'roc_auc')


pipeline  = Pipeline([('feature_sele',rfecv),
                      ('scale', StandardScaler()),
                      ('clf_cv',CV_ada)])

pipeline.fit(X_train, y_train)
y_pred=pipeline.predict(X_test)
print(classification_report(y_test, y_pred))

In [ ]:
cm = confusion_matrix(y_test, y_pred)
disp = ConfusionMatrixDisplay(confusion_matrix=cm)
disp.plot()
plt.show()

### Suport Vector Machines model <a class="anchor" id="mUpSVM"></a>

[TOC](#mtop)

In [ ]:
#this is the classifier used for feature selection
#clf_featr_sele = RandomForestClassifier(n_estimators=10,
#                             random_state=42)

#rfecv = RFECV(estimator=clf_featr_sele,
#              step=1,
#              cv=5,
#              scoring = 'roc_auc')

#you can have different classifier for your final classifier
#clf = svm.SVC()

#CV_svm = GridSearchCV(clf,
#                      param_grid={'C':[0.1, 1.0, 10],
#                                  'kernel':['linear','rbf']},
#                      cv= 5,
#                      scoring = 'roc_auc')


#pipeline  = Pipeline([('feature_sele',rfecv),
#                      ('scale', StandardScaler()),
#                      ('clf_cv',CV_svm)])

#pipeline.fit(X_train, y_train)
#y_pred=pipeline.predict(X_test)
#print(classification_report(y_test, y_pred))

### Artificial Neural Networks <a class="anchor" id="mUpANN"></a>

[TOC](#mtop)

In [ ]:
class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.hidden1 = nn.Linear(rfecv.n_features_, 20)
        self.act1 = nn.ReLU()
        self.hidden2 = nn.Linear(20, 12)
        self.act2 = nn.ReLU()
        self.output = nn.Linear(12, 1)
        self.act_output = nn.Sigmoid()

    def forward(self, x):
        x = self.act1(self.hidden1(x))
        x = self.act2(self.hidden2(x))
        x = self.act_output(self.output(x))
        return x

model = Net()
print(model)

In [ ]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
model.to(device)

In [ ]:
loss_fn = nn.BCELoss()  # binary cross entropy
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [ ]:
scaler = StandardScaler().fit(X_train[X_train.columns[rfecv.support_]])
X_tr_scaled = scaler.transform(X_train[X_train.columns[rfecv.support_]])
X_te_scaled = scaler.transform(X_test[X_test.columns[rfecv.support_]])

In [ ]:
X_train_nn = torch.tensor(X_tr_scaled, dtype=torch.float32).to(device)
y_train_nn = torch.tensor(y_train.to_numpy(), dtype=torch.float32).reshape(-1, 1).to(device)

In [ ]:
n_epochs = 25 #50
batch_size = 10

for epoch in range(n_epochs):
    for i in range(0, len(X_train_nn), batch_size):
        Xbatch = X_train_nn[i:i+batch_size]
        y_pred = model(Xbatch)
        ybatch = y_train_nn[i:i+batch_size]
        loss = loss_fn(y_pred, ybatch)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    print(f'Finished epoch {epoch}, latest loss {loss}')

In [ ]:
X_test_nn = torch.tensor(X_te_scaled, dtype=torch.float32).to(device)
y_test_nn = torch.tensor(y_test.to_numpy(), dtype=torch.float32).reshape(-1, 1).to(device)

In [ ]:
with torch.no_grad(): y_pred = model(X_test_nn)
accuracy = (y_pred.round() == y_test_nn).float().mean()
print(f"Accuracy {accuracy}")

In [ ]:
y_pred=pd.DataFrame(y_pred.round().cpu().detach().numpy())
print(f"Precision {precision_score(y_test, y_pred, pos_label=1)}")
print(f"Recall {recall_score(y_test, y_pred, pos_label=1)}")
print(f"F1-score {f1_score(y_test, y_pred, pos_label=1)}")

In [ ]:
print(f"Precision {precision_score(y_test, y_pred, pos_label=0)}")
print(f"Recall {recall_score(y_test, y_pred, pos_label=0)}")
print(f"F1-score {f1_score(y_test, y_pred, pos_label=0)}")

<table>
    <thead>
        <tr>
            <th>Models</th>
            <th>Accuracy</th>
            <th colspan=2 align=center>Precision</th>
            <th colspan=2 align=center>Recall</th>
            <th colspan=2 align=center>F1-score</th>
        </tr>
    </thead>
    <tbody>
        <tr>
            <td></td>       
            <td></td>
            <td>Legitimate</td>
            <td>Fraud</td>
            <td>Legitimate</td>
            <td>Fraud</td>
            <td>Legitimate</td>
            <td>Fraud</td>
        </tr>
        <tr>
            <td align=right>Decision trees</td>
            <td  align=center>3</td>            
            <td align=center>sss</td>
            <td align=center>zz</td>
            <td align=center>aa</td>
            <td align=center>ss</td>
            <td align=center>ss</td>
            <td align=center>ss</td>
        </tr>
        <tr>
            <td align=right>Random forests</td>
            <td align=center>qq</td>
            <td align=center>ww</td>
            <td align=center>zz</td>
            <td align=center></td>
            <td align=center>ss</td>
            <td align=center>dd</td>
            <td align=center>ss</td>
        </tr>
        <tr>
            <td align=right>Adaboost</td>
            <td align=center>ll</td>
            <td align=center>xx</td>
            <td align=center>zz</td>
            <td align=center></td>
            <td align=center>ss</td>
            <td align=center></td>
            <td align=center>ss</td>
        </tr>
        <tr>
            <td align=right>Suport vector machines</td>
            <td align=center>ll</td>
            <td align=center>oo</td>
            <td align=center>zz</td>
            <td align=center>dd</td>
            <td align=center>ss</td>
            <td align=center></td>
            <td align=center>ss</td>
        </tr>
        <tr>
            <td align=right>Artificial neural networks</td>
            <td align=center>L2</td>
            <td align=center>L3</td>
            <td align=center>zz</td>
            <td align=center></td>
            <td align=center>ss</td>
            <td align=center></td>
            <td align=center>ss</td>
        </tr>
    </tbody>
</table>

## Conclusions <a class="anchor" id="mconclusions"></a>

[TOC](#mtop)

The data was investigated by checking for data unbalancing, visualizing the features, checking the null values, and understanding the relationship between different features. The data were split into train and test sets and scaled, and a feature selection technique based on feature importance was employed and evaluated for use in further pipelines. The hyperparameters were prepared for optimization, and a pipeline was created for each model with feature selection, scaling, hyperparameters tunning, and classification steps. The performance of Decision Trees, Random Forests, and Support Vector Machines are very similar. However, Adaboost ensemble machine learning models present the general best performance compared to all analyzed models. Another work that will be completed in the future is using neural nets to see if we could further improve the model results.